In [130]:
from decouple import config
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker
from operator import itemgetter

from langchain.chains import create_sql_query_chain, LLMChain
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain.evaluation import load_evaluator, EmbeddingDistance
from langchain_community.embeddings import HuggingFaceEmbeddings

import psycopg2
import json

In [2]:
OPENAI_API_KEY = config("OPENAI_API_KEY")
DB_USER = config('DB_USER')
DB_PASSWORD = config('DB_PASSWORD')
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = "food_security"
SAMPLE_QUESTIONS = {"low-birthweight": "Which region has the highest number of children born with low birth weights?",
                    "vaccine_rates": "Which vaccine has the lowest vaccination percentage?",
                    "vaccine_rates_all": "What percentage of children received all vaccines before 12 months",
                    "kids_in_sch": "Whats average percentage of children who are in preschool",
                    "vaccines": "which vaccines did children get in Tunisia?"
                    }
                    
# Create the database URL
DATABASE_URL = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
FILE_SQL_EXAMPLES_EN = "sql_examples_en.json"
USE_BEST_MATCHING_COLUMNS = False

## Prompt Templates

### Create Prompt to Select best Table

In [3]:
def connect_to_database(database_url=DATABASE_URL):
    """Connects to a postgreSQL


    Parameters
    ----------
    database_url : String
        postgreSQL database connection URL, by default DATABASE_URL
    """
    # conn = psycopg2.connect(f"dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD}")
    conn = psycopg2.connect(database_url)

    cur = conn.cursor()

    # Query to get table names and column names
    cur.execute("SELECT table_name, description FROM table_metadata")
    tables = cur.fetchall()

    cur.execute("SELECT table_name, column_name, description FROM column_metadata")
    columns = cur.fetchall()

    cur.close()
    conn.close()

    return tables, columns

In [4]:
def find_best_table_prompt(user_query, tables, columns, 
                           return_chain=True, llm=None):# Define the template for selecting the best table
    template = """
    You are a database assistant. Given the following tables and columns with their descriptions, select the best table that matches the user's query.

    Tables and Columns:
    {table_info}

    User Query:
    {user_query}

    Provide the output in the following JSON format:
    {{
        "best_matching_table": {{
            "table_name": "<best_table_name>",
            "description": "<best_table_description>"
        }}
    }}
    """
    # Prepare the table_info string including descriptions for each table and its columns
    table_info = ""
    for table in tables:
        table_name, table_description = table
        table_info += f"Table: {table_name} - {table_description}\n"
        table_columns = [col for col in columns if col[0] == table_name]
        for column in table_columns:
            _, column_name, column_description = column
            table_info += f"    Column: {column_name} - {column_description}\n"
        table_info += "\n"

    # Create the PromptTemplate
    prompt_template = PromptTemplate(
        template=template,
        input_variables=["table_info", "user_query"]
    )

    # Format the template 
    formatted_prompt = prompt_template.format(table_info=table_info, user_query=user_query)

    if return_chain:
        # Create the chain using the ChatOpenAI model and the PromptTemplate
        chain = LLMChain(llm=llm,prompt=prompt_template)
        return chain, {"table_info": table_info, "user_query": user_query}

    return formatted_prompt


In [5]:
def get_columns_info(table_name, columns):
    columns_info = ""
    for column in columns:
        table, column_name, column_description = column
        if table == table_name:
            columns_info += f"    Column: {column_name} - {column_description}\n"
    return columns_info


In [6]:
def find_best_columns_prompt(user_query, best_matching_table, columns, 
                       return_chain=True, llm=None):
    # Define the template for selecting the relevant columns
    column_template = """
    You are a database assistant. Given the following columns for the table '{table_name}', select the columns that are most relevant to the user's query.

    Table Description: {table_description}

    Columns:
    {columns_info}

    User Query:
    {user_query}

    Relevant Columns:
    """

    columns_info = get_columns_info(best_matching_table["table_name"], columns)

    # Create the PromptTemplate for column selection
    column_prompt_template = PromptTemplate(
        template=column_template,
        input_variables=["table_name", "table_description", "columns_info", "user_query"]
    )

    # Example usage of the template with a user query
    formatted_column_prompt = column_prompt_template.format(
        table_name=best_matching_table["table_name"],
        table_description=best_matching_table["description"],
        columns_info=columns_info,
        user_query=user_query
    )

    # Prepare the context for running the chain
    context = {
        "table_name": best_matching_table["table_name"],
        "table_description": best_matching_table["description"],
        "columns_info": columns_info,
        "user_query": user_query}

    if return_chain:
        chain = LLMChain(llm=llm,prompt=column_prompt_template)
        return chain, context

    return formatted_column_prompt

In [7]:
def load_sql_examples(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

In [20]:
def create_sql_prompt(examples, best_matching_table, columns_metadata, 
                      use_best_matching_columns=False, llm=None):
    """
    Creates a FewShotPromptTemplate for generating SQL queries based on table and column metadata.

    This function generates a prompt template that includes detailed information about the table and its columns.
    The generated prompt instructs a language model (LLM) to create a syntactically correct SQL query based on
    user input. If the table contains a date column and the user does not specify a date, the prompt also instructs
    the LLM to retrieve the most recent data available.

    Parameters
    ----------
    examples : list of dict
        A list of example inputs and corresponding SQL queries. Each example should be a dictionary with 'input' and 'query' keys.
    best_matching_table : dict
        A dictionary containing the best matching table information with 'table_name' and 'description' keys.
    columns_metadata : list of tuples
        A list of tuples containing columns metadata. Each tuple should include 'table_name', 'column_name', and 'description'.
    use_best_matching_columns : bool, optional
        A flag indicating whether to use only the best-matching columns (if True) or all columns in the table (if False). Default is True.

    Returns
    -------
    sql_prompt : FewShotPromptTemplate
        A FewShotPromptTemplate object that can be used with an LLM to generate SQL queries.
    """
    # Prepare table_info string based on the best matching table and columns
    # table_info = f"Table: {best_matching_table['table_name']} - {best_matching_table['description']}\n"
    columns_info = "Columns:\n"
    has_date_column = False

    # Determine which columns to use: best-matching or all columns
    if use_best_matching_columns:
        # If using best_matching_columns, use those provided (filtering columns_metadata based on matching logic)
        columns_to_use = columns_metadata  # Assuming columns_metadata is already filtered
    else:
        # Use all columns for the given table from columns_metadata
        columns_to_use = [col for col in columns_metadata if col[0] == best_matching_table['table_name']]

    # Construct the columns_info string
    for column in columns_to_use:
        table_name, column_name, column_description = column
        columns_info += f"    Column: {column_name} - {column_description}\n"
        if 'date' in column_name.lower():
            has_date_column = True

    # Create FewShot Prompt with instructions for handling most recent data
    example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")

    # Add a special instruction if the table has a date column
    recent_data_instruction = (
        "If the user does not specify a date, retrieve the most recent data available by ordering the results "
        "by the date column in descending order."
    ) if has_date_column else ""

    # Combine table_info and columns_info in the prompt
    sql_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix=(
            "You are a PostgreSQL expert. Given an input question, create a syntactically correct PostgreSQL query to run. "
            "Unless otherwise specified, do not return more than {top_k} rows.\n\n"
            "Here is the relevant table information:\n{table_info}\n\n"
            f"{recent_data_instruction}\n\n"
            "Below are a number of examples of questions and their corresponding SQL queries."
        ),
        suffix="User input: {input}\nSQL query: ",
        input_variables=["input", "table_info", "top_k"],
    )
    
    
    return sql_prompt

In [30]:
def create_answer_chain(llm):
    # Define the prompt template with emphasis on including units, time-specific details, and using the latest data when time is not specified
    answer_prompt = PromptTemplate.from_template(
        """
        You are a knowledgeable assistant. Given the following user question and SQL result, answer the question accurately.
        
        Always ensure to:
        1. Include appropriate units in your answer (e.g., Kwacha per kg, liters, etc.).
        2. Specify the time period or date if the question implies or explicitly asks for it.
        3. If the user does not specify a time, provide the most recent information available in the database and clearly state that this is the latest data.

        For example, if the user asks "What's the price of Maize?", your answer should include the price with the correct unit and mention that this is the most recent price, e.g., "The most recent price of Maize is 60 Kwacha per kg."
        If the user asks about a specific time period, such as "What's the price of Maize for May 2024?", include the time in your answer, e.g., "The price of Maize in May 2024 is 60 Kwacha per kg."

        Question: {question}
        SQL Result: {result}
        Answer: """
    )

   
    # Create answer chain
    return answer_prompt | llm | StrOutputParser()

In [32]:
user_question = "Whats the price of Maize in Rumphi?"
llm = ChatOpenAI()
examples = load_sql_examples(file_path=FILE_SQL_EXAMPLES_EN)

# Retrieve the metadata info (tables and columns)
tables, columns = connect_to_database()

# Chain 1: Find the Best Table
best_table_chain, context = find_best_table_prompt(user_question, tables, 
                                                       columns, llm=llm)
best_table_output_str = best_table_chain.run(**context)

# Convert the string output to a dictionary
try:
    best_table_output = json.loads(best_table_output_str)['best_matching_table']
except json.JSONDecodeError:
    print("Error: The output is not valid JSON.")
    best_table_output = None

# Chain 2: Find Relevant Columns
best_columns_chain, context = find_best_columns_prompt(user_question, best_table_output, 
                                                        columns, llm=llm)
best_columns_output = best_columns_chain.run(**context)

# Create SQL Query
if USE_BEST_MATCHING_COLUMNS:
        sql_prompt = create_sql_prompt(examples=examples, best_matching_table=best_table_output, 
                                   columns_metadata=best_columns_output, 
                                   use_best_matching_columns=True)
else:
        sql_prompt = create_sql_prompt(examples=examples, best_matching_table=best_table_output, 
                                   columns_metadata=columns)


# Initialize LLM and other components
best_table = best_table_output['table_name']
engine = create_engine(DATABASE_URL)
db = SQLDatabase(engine=engine, ignore_tables=['table_metadata', 'column_metadata'])

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db, sql_prompt)

# Create the answer chain
answer_chain = create_answer_chain(llm) | StrOutputParser()

# Put everything together
master_chain = (
RunnablePassthrough.assign(query=write_query).assign(
            result=itemgetter("query") | execute_query
        )
        | answer_chain
    )

response = master_chain.invoke({"question":user_question, 
                                "top_k":3, "table_info":best_table}) 


## Evaluate results 

In [123]:
df_eval = pd.read_csv("data/raw/eval-set.csv")

In [132]:
# Setup evaluator
embedding_model = HuggingFaceEmbeddings()
hf_evaluator = load_evaluator("embedding_distance", distance_metric=EmbeddingDistance.COSINE, 
                              embeddings=embedding_model)

/Users/dunstanmatekenya/Library/CloudStorage/GoogleDrive-dmatekenya@gmail.com/My Drive/phosoAI-whatsapp-app-aws/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/Users/dunstanmatekenya/Library/CloudStorage/GoogleDrive-dmatekenya@gmail.com/My Drive/phosoAI-whatsapp-app-aws/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def evaluate_english_questions(df, prediction_chain):
    for idx, row in df.iterrows():
        question = row['question']
        reference = row['response-1']
        prediction = master_chain.invoke
    

In [63]:
import pandas as pd
df = pd.read_csv("data/tables/prices.csv")

In [117]:
district = "Balaka"
month = "May"
crop = "Maize"
query = f'District == "{district}" and Month_Name == "{month}" and Commodity == "{crop}"'

result = df.query(query)
result.Price.mean()

633.265

In [120]:
df_ny = pd.read_json("sql_examples_ny.json")

In [122]:
df_ny.to_csv("tmp_ny_questions.csv")

In [86]:
df.query('Commodity == "Beans" and Month_Name == "May"').Price.mean()

2704.912755319149

In [87]:
df_prod = pd.read_csv("data/tables/production.csv")

In [103]:
df_prod.query('Crop == "Maize"').sort_values(by=["Yield", "District"], ascending=False)

,District,Crop,Yield,Season
10,Lilongwe,Maize,444440.000,2023-2024
8,Kasungu,Maize,318203.000,2023-2024
6,Dowa,Maize,259890.000,2023-2024
15,Mzimba,Maize,248415.253,2023-2024
13,Mchinji,Maize,244792.000,2023-2024
5,Dedza,Maize,227724.000,2023-2024
21,Ntchisi,Maize,185021.000,2023-2024
26,Zomba,Maize,119523.000,2023-2024
12,Mangochi,Maize,104262.600,2023-2024
4,Chitipa,Maize,102319.000,2023-2024


In [ ]:
def run_sql_chain(user_query, best_table_info, columns_info, 
                  best_columns=None, llm=None):
    
    # Load examples and create prompts
    examples = load_sql_examples(file_path=FILE_SQL_EXAMPLES_EN)
    if USE_BEST_MATCHING_COLUMNS:
        sql_prompt = create_sql_prompt(examples=examples, best_matching_table=best_table_info, 
                                   columns_metadata=columns_info, 
                                   use_best_matching_columns=True)
    else:
        sql_prompt = create_sql_prompt(examples=examples, best_matching_table=best_table_info, 
                                   columns_metadata=columns_info)
    
    # Initialize LLM and other components
    engine = create_engine(DATABASE_URL)
    db = SQLDatabase(engine=engine, ignore_tables=['table_metadata', 'column_metadata'])

    execute_query = QuerySQLDataBaseTool(db=db)
    write_query = create_sql_query_chain(llm, db, sql_prompt)

    # Create the answer chain
    answer_chain = create_answer_chain(llm)

    # Put everything together
    chain = (
        RunnablePassthrough.assign(query=write_query).assign(
            result=itemgetter("query") | execute_query
        )
        | answer_chain
    )

    return chain.invoke({"question": "{}".format(user_question)})

In [ ]:
def process_sql_query(user_question, llm=None):
    
    # LLM
    if not llm:
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

    # Retrieve the metadata info (tables and columns)
    tables, columns = connect_to_database()

    # Chain 1: Find the Best Table
    best_table_chain, context = find_best_table_prompt(user_question, tables, 
                                                       columns, llm=llm)
    best_table_output_str = best_table_chain.run(**context)

    # Convert the string output to a dictionary
    try:
        best_table_output = json.loads(best_table_output_str)
    except json.JSONDecodeError:
        print("Error: The output is not valid JSON.")
        best_table_output = None

    # Chain 2: Find Relevant Columns
    table_name = best_table_output['best_matching_table']['table_name']
    best_columns_chain, context = find_best_columns_prompt(user_question, best_table_output['best_matching_table'], 
                                                        columns, llm=llm)
    best_columns_output = best_columns_chain.run(**context)
    

    response = run_sql_chain(user_question, best_table_output['best_matching_table'], 
                            columns_info, best_columns_output, llm=llm)
    

    return response

    

In [ ]:
question = "Whats the price of Maize?"
openai_chat_model = ChatOpenAI()
response = process_sql_query(user_question=question, llm=openai_chat_model)


create_sql_prompt(examples, best_matching_table, columns_metadata)

In [ ]:
response

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(temperature=0.5)
messages = [SystemMessage(content='''Act as a senior software engineer
at a startup company.'''),
HumanMessage(content='''Please can you provide a funny joke
about software engineers?''')]
response = chat.invoke(input=messages)
print(response.content)

In [ ]:
chat = ChatOpenAI(temperature=0.5)
messages = [SystemMessage(content='''You are a highly-skilled linguist and polyglot.\
              Identify the language of the user query'''),
HumanMessage(content='''Dzina langa ndine Dunstan Matekenya''')]
response = chat.invoke(input=messages)
print(response.content)

In [ ]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

In [ ]:
# 2x lists of messages, which is the same as [messages, messages]
synchronous_llm_result = chat.batch([messages]*2)
print(synchronous_llm_result)

In [ ]:
from langchain_core.runnables.config import RunnableConfig

# Create a RunnableConfig with the desired concurrency limit:
config = RunnableConfig(max_concurrency=5)

# Call the .batch() method with the inputs and config:
results = chat.batch([messages, messages], config=config)

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import (SystemMessagePromptTemplate,
ChatPromptTemplate)

template = """
You are a creative consultant brainstorming names for businesses.

You must follow the following principles:
{principles}

Please generate a numerical list of five catchy names for a start-up in the
{industry} industry that deals with {context}?

Here is an example output format:
- Name1
- Name2
- Name3
"""


model = ChatOpenAI()
system_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_prompt])

chain = chat_prompt | model

result = chain.invoke({
    "industry": "medical",
    "context":'''creating AI solutions by automatically summarizing patient
    records''',
    "principles":'''1. Each name should be short and easy to
    remember. 2. Each name should be easy to pronounce.
    3. Each name should be unique and not already taken by another company.'''
})

print(result.content)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate
from langchain_openai import ChatOpenAI

# Create the template text
template = '''You are a helpful assistant that translates {input_language}
to {output_language}.'''

# Create the PromptTemplate Object 
prompt = PromptTemplate(
    template = template,
    input_variables = ["input_language", "output_language"]
)

# Convert base prompt into Chat System prompt 
system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

# Create chat model
chat = ChatOpenAI()

# Create message from user
text_to_translate = HumanMessage(content="Chimanga chikupezeka kuti?")

# format the system message
formatted_sys_message = system_message_prompt.format(
    input_language="Chichewa", output_language="English")

response = chat.invoke([formatted_sys_message, text_to_translate] )


In [ ]:
SystemMessagePromptTemplate.format_messages?

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessage
from langchain_openai import ChatOpenAI

# Define the system message template
template = '''You are a helpful assistant that translates {input_language} to {output_language}.'''

prompt = PromptTemplate(
    template=template,
    input_variables=["input_language", "output_language"]
)

# Create the system message prompt
system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

# Format the system message
formatted_system_message = system_message_prompt.format(input_language="Chichewa", output_language="English")

# Create a human message
text_to_translate = HumanMessage(content="Chimanga chikupezeka kuti?")

# Initialize the chat model
chat = ChatOpenAI()

# Call the invoke method with both the human message and the formatted system message
response = chat.invoke([formatted_system_message, text_to_translate])

print(response)


## Output Parsers 
Enables you to take outputs from an LLM and convert it into the format you need.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel, Field
from typing import List


temperature = 0.0

class BusinessName(BaseModel):
    name: str = Field(description="The name of the business")
    rating_score: float = Field(description='''The rating score of the
    business. 0 is the worst, 10 is the best.''')

class BusinessNames(BaseModel):
    names: List[BusinessName] = Field(description='''A list
    of busines names''')


# Set up a parser + inject instructions into the prompt template:
parser = PydanticOutputParser(pydantic_object=BusinessNames)

principles = """
- The name must be easy to remember.
- Use the {industry} industry and Company context to create an effective name.
- The name must be easy to pronounce.
- You must only return the name without any other text or characters.
- Avoid returning full stops, \n, or any other characters.
- The maximum length of the name must be 10 characters.
"""

# Chat Model Output Parser:
model = ChatOpenAI()
template = """Generate five business names for a new start-up company in the
{industry} industry.
You must follow the following principles: {principles}
{format_instructions}
"""


system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

In [ ]:
AIMessage(content='''Vous êtes un assistant utile qui traduit l'anglais en
français.''', additional_kwargs={}, example=False)

In [ ]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """


In [ ]:
from langchain_openai import OpenAI

# initialize the models
openai = OpenAI(openai_api_key=OPENAI_API_KEY)


In [ ]:
print(openai(prompt))


In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [ ]:
openai(formatted_prompt)

In [ ]:
formatted_prompt = prompt_template.format(query="Which libraries and model providers offer LLMs?")

In [ ]:
type(prompt_template)

In [ ]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. Here are some examples: 

User: What is the meaning of life?
AI: """

openai.temperature = 1.0  # increase creativity/randomness of output

print(openai(prompt))


In [ ]:
prompt = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """

print(openai(prompt))


In [ ]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)


In [ ]:
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))


In [ ]:
openai(few_shot_prompt_template.format(query=query))

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.tools import langdetect

In [ ]:
!pip install pandas openai

In [ ]:
import pandas as pd
from openai import OpenAI
import os

# Define two variants of the prompt to test zero-shot
# vs few-shot
prompt_A = """Product description: A pair of shoes that can
fit any foot size.
Seed words: adaptable, fit, omni-fit.
Product names:"""

prompt_B = """Product description: A home milkshake maker.
Seed words: fast, healthy, compact.
Product names: HomeShaker, Fit Shaker, QuickShake, Shake
Maker

Product description: A watch that can tell accurate time in
space.
Seed words: astronaut, space-hardened, eliptical orbit
Product names: AstroTime, SpaceGuard, Orbit-Accurate,
EliptoTime.

Product description: A pair of shoes that can fit any foot
size.
Seed words: adaptable, fit, omni-fit.
Product names:"""

test_prompts = [prompt_A, prompt_B]


# Set your OpenAI key as an environment variable
# https://platform.openai.com/api-keys
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # Default
)

def get_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return response.choices[0].message.content

# Iterate through the prompts and get responses
responses = []
num_tests = 5

for idx, prompt in enumerate(test_prompts):
    # prompt number as a letter
    var_name = chr(ord('A') + idx)

    for i in range(num_tests):
        # Get a response from the model
        response = get_response(prompt)

        data = {
            "variant": var_name,
            "prompt": prompt,
            "response": response
            }
        responses.append(data)

# Convert responses into a dataframe
df = pd.DataFrame(responses)

# Save the dataframe as a CSV file
df.to_csv("responses.csv", index=False)

print(df)

In [ ]:
df

In [ ]:
!pip install ipywidgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# load the responses.csv file
df = pd.read_csv("responses.csv")

# Shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

# df is your dataframe and 'response' is the column with the
# text you want to test
response_index = 0
# add a new column to store feedback
df['feedback'] = pd.Series(dtype='str')


def update_response():
    new_response = df.iloc[response_index]['response']
    if pd.notna(new_response):
        new_response = "<p>" + new_response + "</p>"
    else:
        new_response = "<p>No response</p>"
    response.value = new_response
    count_label.value = f"Response: {response_index + 1}"
    count_label.value += f"/{len(df)}"


In [ ]:
def on_button_clicked(b):
    global response_index
    #  convert thumbs up / down to 1 / 0
    user_feedback = 1 if b.description == "\U0001F44D" else 0

    # update the feedback column
    df.at[response_index, 'feedback'] = user_feedback

    response_index += 1
    if response_index < len(df):
        update_response()
    else:
        # save the feedback to a CSV file
        df.to_csv("results.csv", index=False)

        print("A/B testing completed. Here's the results:")
        # Calculate score and num rows for each variant
        summary_df = df.groupby('variant').agg(
            count=('feedback', 'count'),
            score=('feedback', 'mean')).reset_index()
        print(summary_df)


In [ ]:
response = widgets.HTML()
count_label = widgets.Label()

update_response()

thumbs_up_button = widgets.Button(description='\U0001F44D')
thumbs_up_button.on_click(on_button_clicked)

thumbs_down_button = widgets.Button(
    description='\U0001F44E')
thumbs_down_button.on_click(on_button_clicked)

button_box = widgets.HBox([thumbs_down_button,
thumbs_up_button])

display(response, button_box, count_label)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# load the responses.csv file
df = pd.read_csv("responses.csv")

# Shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

# df is your dataframe and 'response' is the column with the
# text you want to test
response_index = 0
# add a new column to store feedback
df['feedback'] = pd.Series(dtype='str')

def on_button_clicked(b):
    global response_index
    #  convert thumbs up / down to 1 / 0
    user_feedback = 1 if b.description == "\U0001F44D" else 0

    # update the feedback column
    df.at[response_index, 'feedback'] = user_feedback

    response_index += 1
    if response_index < len(df):
        update_response()
    else:
        # save the feedback to a CSV file
        df.to_csv("results.csv", index=False)

        print("A/B testing completed. Here's the results:")
        # Calculate score and num rows for each variant
        summary_df = df.groupby('variant').agg(
            count=('feedback', 'count'),
            score=('feedback', 'mean')).reset_index()
        print(summary_df)

def update_response():
    new_response = df.iloc[response_index]['response']
    if pd.notna(new_response):
        new_response = "<p>" + new_response + "</p>"
    else:
        new_response = "<p>No response</p>"
    response.value = new_response
    count_label.value = f"Response: {response_index + 1}"
    count_label.value += f"/{len(df)}"

response = widgets.HTML()
count_label = widgets.Label()

update_response()

thumbs_up_button = widgets.Button(description='\U0001F44D')
thumbs_up_button.on_click(on_button_clicked)

thumbs_down_button = widgets.Button(
    description='\U0001F44E')
thumbs_down_button.on_click(on_button_clicked)

button_box = widgets.HBox([thumbs_down_button,
thumbs_up_button])

In [ ]:
display(response, button_box, count_label)

In [ ]:
from langchain.tools import LangDetectTool

In [ ]:
!pip install langdetect

In [ ]:
def detect_language(text):
     # Initialize the OpenAI API
    llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-3.5-turbo')

    # Create the language detection tool
    lang_detect_tool = LangDetectTool()

    # Create a Tool object
    lang_detect = Tool(
        name="Language Detection",
        func=lang_detect_tool.run,
        description="Useful for detecting the language of a given text."
    )

    # Use the tool to detect language
    return lang_detect.run(text)
                             

In [ ]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(query):
    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Similarity results=>", most_similar)
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


In [ ]:
prompt_router("Explain Pythagoras theorem")

In [ ]:
def create_db_object_with_metadata():
    # Create the SQLAlchemy engine
    engine = create_engine(DATABASE_URL)
    metadata_obj = MetaData()
    metadata_obj.reflect(bind=engine)

    # Create a configured "Session" class
    Session = sessionmaker(bind=engine)
    session = Session()

    # Load custom metadata from the table_metadata and column_metadata tables
    try:
        table_metadata = session.execute("SELECT * FROM table_metadata").fetchall()
        column_metadata = session.execute("SELECT * FROM column_metadata").fetchall()

        # Add table metadata
        for row in table_metadata:
            print(row)
            table_name = row['table_name']
            description = row['description']
            table = metadata_obj.tables.get(table_name)
            table.info['description'] = description

        # Add column metadata
        for row in column_metadata:
            table_name = row['table_name']
            column_name = row['column_name']
            description = row['description']
            table = metadata_obj.tables.get(table_name)
            column = table.columns.get(column_name)
            column.info['description'] = description
    finally:
        session.close()
    db = SQLDatabase(engine=engine, metadata=metadata_obj, ignore_tables=['table_metadata', 'column_metadata'])

    return db

In [ ]:
commodities_price = ['Maize', 'Rice', 'Soya beans', 'Beans', 'Cow peas', 'Groundnuts']
crop_estimates = ['Maize', 'Beans', 'Cow peas', 'Dolichus beans ', 'Soy beans',
       'Ground beans', 'Paprika', 'Rice', 'Pigeon peas', 'Grams',
       'Sesame ', 'Field peas', 'Velvet beans', 'Chick peas', 'Wheat',
       'Millet', 'Sorghum ', 'Groundnuts', 'Cassava', 'Sweet potatoes',
       'Potatoes', 'Tobacco', 'Flue cured', 'Sunflower ', 'Chillies',
       'Cotton ', 'Bananas', 'Mangoes', 'Oranges', 'Tangerines', 'Coffee',
       'Pineapples', 'Guava', 'Pawpaws', 'Peaches', 'Lemons',
       'Grape fruits', 'Apples', 'Avocado pear', 'Macademia', 'Tomatoes',
       'Onions', 'Cabbage', 'Egg plants', 'Okra', 'Cucumber']
price_estimates_key_words = ["price", "cheap", "produce", 
                                 "buy", "sell", "sale", "find"]
all_kw = [i.lower() for i in set(commodities_price+crop_estimates+price_estimates_key_words)]
print(all_kw)

In [ ]:
import os
import json
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

def load_examples(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)



def format_translation_examples(examples_file, source_language, target_language):
    examples = load_examples(examples_file)
    key = f"{source_language}-{target_language}"
    if key in examples:
        return "\n".join([f"{source_language}: {ex[source_language]}\n{target_language}: {ex[target_language]}" 
                          for ex in examples[key]])
    return ""


def translate_with_openai(text, src_lan, dest_lan):
    # Create a ChatOpenAI instance
    chat_model = ChatOpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)
    
    # Get and format translation examples
    formated_examples = format_translation_examples("./translation_examples.json", source_language=src_lan, 
                                target_language=dest_lan)
    # Create a system message with examples
    system_template = """You are a professional translator. Your task is to translate {src_lan} to {dest_lan}.
        Here are a few examples:

        {examples}

        Now, translate the following text:"""

    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

    # Create a human message for the actual translation request
    human_template = "{text}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    # Combine the prompts
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    # Create an LLMChain for translation
    translation_chain = LLMChain(llm=chat_model, prompt=chat_prompt)
    
    return translation_chain.run({
        "source_language": src_lan,
        "target_language": dest_lan,
        "examples": formated_examples,
        "text": text
    })



In [ ]:
chat_model = ChatOpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)
# Create a system message with examples
system_template = """You are a professional translator. Your task is to translate {source_language} to {target_language}.
Here are a few examples:

{examples}

Now, translate the following text:"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# Create a human message for the actual translation request
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Combine the prompts
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Create an LLMChain for translation
translation_chain = LLMChain(llm=chat_model, prompt=chat_prompt)

# Function to translate text
def translate_text(text, source_language, target_language):
    formated_examples = format_translation_examples("./translation_examples.json", source_language, 
                                target_language)
    return translation_chain.run({
        "source_language": source_language,
        "target_language": target_language,
        "examples": formated_examples,
        "text": text
    })

In [ ]:
translate_text("Mpunga ukugulitsidwa pa mtengo wanji?", "Chichewa", "English")

In [ ]:
format_translation_examples(examples_file="./translation_examples.json", 
                            source_language="Chichewa", target_language="English")

In [ ]:
format_examples(examples, source_language="Chichewa", target_language="English")

In [ ]:
relevant_examples = examples["Chichewa-English"]

for item in relevant_examples:
    print(item)
    "\n".join([f"{source_language}: {ex['source']}\n{target_language}: {ex['target']}" 
                          for ex in examples[key]])

In [ ]:


# Create a ChatOpenAI instance
chat_model = ChatOpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)

# Create a system message with examples
system_template = """You are a professional translator. Your task is to translate {source_language} to {target_language}.
Here are a few examples:

{source_language}: Mtedza ukugulitsidwa pabwanji?
{target_language}: Whats the price of groundnuts?

{source_language}: Chimanga chikupezeka kuti?
{target_language}: Where can I find maize?

{source_language}: Ndikuti nyemba zikutchipa?
{target_language}: Where can I find beans at cheap price?

{source_language}: Chimanga chili pabwanji pano?
{target_language}: Whats the price of maize now?

{source_language}: Ku Dowa chimanga chili pa bwanji?
{target_language}: Whats the price of maize in Dowa?

{source_language}: Kodi ndi boma liti anakolola chimanga chambiri pakati pa Lilongwe kapena Kasungu?
{target_language}: Which district produced more maize: Lilongwe or Kasungu?

{source_language}: Kodi chimanga chili pa bwanji ku Rumphi?
{target_language}: How much is maize per Kg in Rumphi?

{source_language}: Mpunga ukugulitsidwa ndalama zingati ku Lilongwe?
{target_language}: Whats the price of rice in Lilongwe?

{source_language}: Mtedza otchipa ukupezeka mboma liti?
{target_language}: Which district has the cheap price for groundnuts?

{source_language}: Chimanga chambiri chikupezeka kuti?
{target_language}: Where can I find maize?

{source_language}: Ndi boma liti komwe anakolola chimanga chambiri?
{target_language}: Which district harvested large quantities of maize?

{source_language}: Ndi mbeu zanji anakolola bwino ku Rumphi?
{target_language}: Which crops produced the most yields in Rumphi

{source_language}: Soya ali pabwanji?
{target_language}: Whats the price of soya?

{source_language}: Mtedza otchipa ndingaupeze kuti?
{target_language}: Where can I find groundnuts at reasonable price?


Now, translate the following text:"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# Create a human message for the actual translation request
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Combine the prompts
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Create an LLMChain for translation
translation_chain = LLMChain(llm=chat_model, prompt=chat_prompt)

# Function to translate text
def translate_text(text, source_language, target_language):
    return translation_chain.run({
        "source_language": source_language,
        "target_language": target_language,
        "text": text
    })

# Example usage
source_text = "Mtedza ndingaupeze kuti?"
source_language = "Chichewa"
target_language = "English"

translated_text = translate_text(source_text, source_language, target_language)

print(f"{source_language}: {source_text}")
print(f"{target_language}: {translated_text}")

In [ ]:
# Create an OpenAI instance
llm = OpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY, model="gpt-4")

# Create a prompt template for translation
translation_template = PromptTemplate(
    input_variables=["source_language", "target_language", "text"],
    template="Translate the following {source_language} text to {target_language}: {text}"
)

# Create an LLMChain for translation
translation_chain = LLMChain(llm=llm, prompt=translation_template)

# Function to translate text
def translate_text(text, source_language, target_language):
    return translation_chain.run({
        "source_language": source_language,
        "target_language": target_language,
        "text": text
    })

In [ ]:

source_text = "Chimanga chili pa bwanji ku Malawi?"
source_language = "Chichewa"
target_language = "English"

translated_text = translate_text(source_text, source_language, target_language)
print(translated_text)

In [ ]:
translated_text

In [ ]:
from langchain.llms import OpenAI

In [ ]:
def translate_text(text, source_language="English", target_language="Chichewa"):

    llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-3.5-turbo')
    # Create a template for the translation
    translation_template = ChatPromptTemplate.from_template(
    "Translate the following {source_language} text to {target_language}: {text}"
)

    # Create a chain with the LLM and the translation template
    translation_chain = LLMChain(llm=llm, prompt=translation_template)

    translation = translation_chain.run({
        'text': text,
        'source_language': source_language,
        'target_language': target_language
    })
    return translation


In [ ]:
translated_text = translate_text(text="cheap")

In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
import openai
def translate_text(text, source_language="English", target_language="Chichewa"):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": f"You are a helpful assistant that translates {source_language} to {target_language}."
            },
            {
                "role": "user",
                "content": f"Translate the following text from {source_language} to {target_language}:\n\n{text}"
            }
        ]
    )
    translation = response['choices'][0]['message']['content']
    return translation.strip()

In [ ]:
translate_text(text="cheap", source_language="English", target_language="Chichewa")

In [ ]:
from googletrans import Translator
def translate_text(text, source_language="en", target_language="ny"):
    translator = Translator()
    translation = translator.translate(text, src=source_language, dest=target_language)
    return translation.text

In [ ]:
db = create_db_object_with_metadata()
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# chain = create_sql_query_chain(llm, db)
# response = chain.invoke({"question": "{}".format(SAMPLE_QUESTIONS["low-birthweight"])})
# response

In [ ]:
import sqlalchemy

In [ ]:
sqlalchemy.__version__

In [ ]:

examples = [
    {"input": "Which region has the highest number of children born with low birth weights?", 
     "query": "SELECT * FROM tab4711 ORDER BY number_children DESC LIMIT 1;",},

     {"input": "Which region has the highest percentage of children born with low birth weights?", 
     "query": "SELECT * FROM tab4711 ORDER BY percentage_below_2500g DESC LIMIT 1;",
     },

     {"input": "How many children received all vaccines before 12 months?", 
     "query": "SELECT vacc_b4_12months FROM tab501 WHERE vacc_category = 'All vaccinations';"},

     {"input": "Which region has the lowest rates in preschool for children?", 
     "query": "SELECT * FROM tab9011 ORDER BY percentage_children_sch ASC LIMIT 1;",},

     {"input": "Whats the average literacy rate among young women in Tunisia?",
      "query": "SELECT AVG(percentage_literate) AS avg_literacy_rate FROM tab971;",},
]

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "{}".format(SAMPLE_QUESTIONS['vaccine_rates_all'])})

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")
sql_prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a PostgreSQL expert. Given an input question, create a syntactically correct PostgreSQL query to run. Unless otherwise specificed, do not return more than {top_k} rows.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries.",
    suffix="User input: {input}\nSQL query: ",
    input_variables=["input", "top_k", "table_info"],
)

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db, verbose=True)
write_query = create_sql_query_chain(llm, db, sql_prompt)

In [ ]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "{}".format(SAMPLE_QUESTIONS["vaccines"])})